In [1]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
import utils as U
reload(U)
import image_loader as IM
reload(IM)
from theano.sandbox import cuda
from keras.preprocessing import image
%matplotlib inline
import os
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Input, Flatten, Conv2D, AveragePooling2D
from keras.optimizers import Adam
import models.model as M
from keras.layers import Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.layers import Dropout
from keras.regularizers import l2
reload(M)

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


<module 'models.model' from 'models/model.pyc'>

# Pre compute Convolutional Layers

In [2]:
folder = "data/train/"
train_folder = folder+"train"
train_filenames = os.listdir(train_folder)
validation_folder = folder+"valid"
validation_filenames = os.listdir(validation_folder)
batch_size = 64
train_batch_count = len(train_filenames) / batch_size
val_batch_count = len(validation_filenames) / batch_size

In [19]:
gen_t = image.ImageDataGenerator(vertical_flip=True, horizontal_flip=True)

gen = IM.DirectoryIterator(train_folder, train_filenames, image_processor=gen_t, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=False)
val_gen = IM.DirectoryIterator(validation_folder, validation_filenames, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=False)

In [32]:
model = M.full_model(weights_file="weights/best/partly-cloudy-full-training.01-0.25.hdf5")
conv_model = Sequential(model.layers[:-len(M.dense_layers())])

In [34]:
augmented_conv_features = conv_model.predict_generator(gen, train_batch_count, verbose=1)
augmented_val_features = conv_model.predict_generator(val_gen, val_batch_count, verbose=1)

U.save_array("precomputed-partly-cloudy-conv-features", augmented_conv_features)
U.save_array("precomputed-partly-cloudy-conv-val-features", augmented_val_features)

88/88 [==============================] - 157s   


# Train Dense Layers

In [3]:
from utils import binary_one_hot_mapping
augmented_conv_features = U.load_array("precomputed-partly-cloudy-conv-features")
augmented_val_features = U.load_array("precomputed-partly-cloudy-conv-val-features")
batch_size = 64
mapping = binary_one_hot_mapping("cloudy")

train_labels = [ mapping[fname] for fname in train_filenames ]
valid_labels = [ mapping[fname] for fname in validation_filenames ]

In [4]:
dense_layers = M.dense_layers(dropout=0.4, regularization=0.005)

model = Sequential(dense_layers)

In [5]:
augmented_conv_features = U.load_array("precomputed-partly-cloudy-conv-features",)
augmented_val_features = U.load_array("precomputed-partly-cloudy-conv-val-features")

In [6]:
model.load_weights("weights/clear-top-1-epoch")
model.compile(optimizer=(Adam(lr=0.0001)), metrics=["accuracy"], loss="categorical_crossentropy")
model.fit(augmented_conv_features, train_labels, epochs=2, validation_data=(augmented_val_features, valid_labels))

Train on 34816 samples, validate on 5632 samples
Epoch 1/2
34816/34816 [==============================] - 478s - loss: 2.4884 - acc: 0.9351 - val_loss: 1.0350 - val_acc: 0.9329
Epoch 2/2
34816/34816 [==============================] - 478s - loss: 0.7380 - acc: 0.9402 - val_loss: 0.5869 - val_acc: 0.9434


In [7]:
model.save_weights("weights/clear-top-2-epoch")

In [8]:
model.compile(optimizer=(Adam(lr=0.0001)), metrics=["accuracy"], loss="categorical_crossentropy")
model.fit(augmented_conv_features, train_labels, epochs=2, validation_data=(augmented_val_features, valid_labels))

Train on 34816 samples, validate on 5632 samples
Epoch 1/2
34816/34816 [==============================] - 477s - loss: 0.5732 - acc: 0.9418 - val_loss: 0.4855 - val_acc: 0.9371
Epoch 2/2
34816/34816 [==============================] - 477s - loss: 0.5213 - acc: 0.9430 - val_loss: 0.4636 - val_acc: 0.9430


In [9]:
model.save_weights("weights/clear-top-3-epoch")
model.fit(augmented_conv_features, train_labels, epochs=2, validation_data=(augmented_val_features, valid_labels))

Train on 34816 samples, validate on 5632 samples
Epoch 1/2
34816/34816 [==============================] - 477s - loss: 0.4801 - acc: 0.9434 - val_loss: 0.4720 - val_acc: 0.9396
Epoch 2/2
34816/34816 [==============================] - 477s - loss: 0.4584 - acc: 0.9429 - val_loss: 0.4192 - val_acc: 0.9432


In [11]:
model.optimizer.lr = 0.00001
model.fit(augmented_conv_features, train_labels, epochs=2, validation_data=(augmented_val_features, valid_labels))

Train on 34816 samples, validate on 5632 samples
Epoch 1/2
34816/34816 [==============================] - 477s - loss: 0.4074 - acc: 0.9445 - val_loss: 0.4450 - val_acc: 0.9416
Epoch 2/2
34816/34816 [==============================] - 477s - loss: 0.3905 - acc: 0.9456 - val_loss: 0.4398 - val_acc: 0.9281


# Save Full Models weights

In [12]:
model.load_weights("weights/clear-top-3-epoch")
conv_layers = M.vgg().layers
full_model = Sequential(conv_layers + model.layers)
full_model.save_weights("weights/best/no-conv-training/cloudy-acc-.9432-loss-.4192.hdf5")